In [3]:
from posixpath import join
from astropy.io.fits import hdu
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np 
from ciao_contrib.runtool import search_csc 

from astropy.io import fits 
from fields import fields 
from tqdm import tqdm

In [6]:
df = pd.read_csv('../source_list/atnf_catalogue_full.csv' , delimiter=';' , comment= '#' , na_values='*')
df = df.replace(' ' , '_' , regex=True)
df 


,INDEX,NAME,PSRJ,RAJ,DECJ,RAJD,DECJD,BINCOMP,ASSOC,SURVEY,PSR,Unnamed: 11
0,1,J0002+6216,J0002+6216,00:02:58.1,+62:16:09.4,0.742375,62.269278,NaN,"GRS:4FGL_J0002.8+6217[fermi21],XRS:1XSPS",FermiBlind,HE[wcp+18],NaN
1,2,J0006+1834,J0006+1834,00:06:04.8,+18:34:59,1.520000,18.583056,NaN,NaN,ar4,NaN,NaN
2,3,J0007+7303,J0007+7303,00:07:01.7,+73:03:07.4,1.757083,73.052056,NaN,"GRS:4FGL_J0007.0+7303[fermi21],XRS:RX_J0",FermiBlind,NRAD,NaN
3,4,J0011+08,J0011+08,00:11:34,+08:10,2.891667,8.166667,NaN,NaN,ar327,NaN,NaN
4,5,B0011+47,J0014+4746,00:14:17.7,+47:46:33.4,3.573958,47.775944,NaN,NaN,"gb1,gb2,gb3,lotaas",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3172,3173,J2351+8533,J2351+8533,23:51:03,+85:33:20.6,357.763734,85.555735,NaN,NaN,"gbncc,lotaas",NaN,NaN
3173,3174,J2352+65,J2352+65,23:52,+65,358.000000,65.000000,NaN,NaN,"gb350,gbncc,lotaas",NaN,NaN
3174,3175,J2354-22,J2354-22,23:54:26,-22:51:53,358.608333,-22.864722,NaN,NaN,htru_pks,NaN,NaN
3175,3176,B2351+61,J2354+6155,23:54:04.7,+61:55:46.8,358.519929,61.929679,NaN,NaN,"gb1,gb2,gb3,gb4,gbncc,lotaas",NaN,NaN


In [7]:
df['PSR'].value_counts()

RRAT                  80
HE                    71
NRAD                  62
AXP,NRAD              19
HE[sbc+19]            15
                      ..
HE[to07]               1
HE[ncb+15,fermi21]     1
HE[bt93,aaa+09f]       1
HE[hsp+03]             1
HE[rpp+20]             1
Name: PSR, Length: 113, dtype: int64

In [8]:
df['PSR'].unique()

array(['HE[wcp+18]', nan, 'NRAD', 'HE', 'HE[btlb00,aaa+09e]',
       'HE[aaa+10b]', 'RRAT', 'AXP,NRAD', 'RRAT[gac+20]', 'RRAT[ttm18]',
       'RRAT[dsm+16]', 'HE[khvb98,aaa+09f]', 'HE[brj+21]', 'HE[tpc+11]',
       'HE[cck+16]', 'HE[hrm+11,fermi21]', 'HE[trr+21,fermi21]',
       'HE[wlz+18]', 'HE[to07]', 'XINS,NRAD', 'HE[bt93,aaa+09f]',
       'HE[qpl+19,fermi21]', 'HE[brs+19]', 'HE[cdt69,fhm+69,hjm+70]',
       'HE[mkz+03]', 'HE[mp85]', 'HE[egc+13]', 'HE[to05]',
       'HE[tsn+01,waa+10]', 'HE[sbc+19]', 'HE[fermi21]',
       'HE[chmm89,rfk+96,srg+97,w', 'HE[mhl+02,fermi21]',
       'HE[cgm+07,gkj+13]', 'HE[lsg+15,fermi21]', 'NRAD[nie+20]',
       'HE[wob+04,aaa+09f]', 'HE[bwt+04]', 'HE[hg73,tfko75]',
       'HE[dmd+88,fermi21]', 'HE[abb+15,fermi21]', 'HE[mlc+01,fermi21]',
       'HE[ncb+19]', 'HE[ckr+15]', 'HE[wob+04]', 'HE[kbm+03,fermi21]',
       'HE[jah+16,aps+17]', 'HE[aaa+09]', 'HE[cpw+18]', 'HE[drc+21]',
       'HE[klm+00,aaa+09i]', 'HE[aaa+10m]', 'NRAD[fermi21]', 'HE[gkc+07]',


In [62]:
df_select = df[df['Type'].isin(['CataclyV', 'Candidate_CV', 'known_CV', 'N', 'Candidate',
       'Candidate_Nova', 'Known_CV', 'DN', 'AntiNova',
       'Cataclysmic_Variable'])].reset_index(drop=True)

In [63]:
df_select

,Name,Disc. Date,Mag.,R.A.,Dec.,Type
0,1RXS_J013308.9+383218,2009/10/27,NaN,01:33:08.72,+38:32:17.2,CataclyV
1,1RXS_J044027.0+023300,2009/02/19,NaN,04:40:27.14,+02:33:01.0,CataclyV
2,1RXS_J221343.9+173301,2009/09/17,NaN,22:13:44.03,+17:32:51.9,CataclyV
3,2MASS_J00153821+2636574,2009/09/18,NaN,00:15:38.2393,+26:36:56.815,CataclyV
4,2MASS_J23255145-0140232,2009/11/16,NaN,23:25:51.4767,-01:40:23.646,CataclyV
...,...,...,...,...,...,...
11417,ZTF18abwurjm,NaN,19.7,20:07:54.82,+03:27:38.1,DN
11418,ZTF18aceglmy,NaN,19.6,19:49:47.59,+03:13:05.2,DN
11419,ZTF18acrultm,NaN,NaN,05:35:49.47,-03:32:51.3,CataclyV
11420,ZTF19acecsfi,NaN,NaN,07:59:30.81,+44:16:21.5,Candidate_CV


In [50]:
ra = df_select['R.A.']
dec = df_select['Dec.']
name = df_select['Name']
num_obs = [] 
offset = []


In [ ]:
type = df_select['Type']
for index in tqdm(range(len(df_select))):
    try :
        s = search_csc(
                        radunit="arcsec", 
                        columns=fields, 
                        bands="broad",
                        clobber="yes" ,
                        radius= 10 , 
                        pos= str(ra[index])+','+str(dec[index])  ,
                        outfile='temp.csv')
                    #sys('search_csc outfile=trial.csv radunit=arcsec columns="SOS,SOP,SOV , o.gti_obs m.flux_aper_b" bands=broad clobber=yes radius=1 pos="65.428058,32.907468"')
                    #print(s)
                    
        data = pd.read_csv('temp.csv', delimiter='\t' , comment='#')
        data = data[data['match_type']=='u          ']
        data = data[data['instrument']=='ACIS']
        data.index.name= 'index'
        #f_name_fits = data_fits['A_NAME']
        n = len(data)
        num_obs.append(n)
        if(n>0):
            print('Match found --' , len(data))
            data.to_csv('../data/all_data/CV_v2_all/'+name[index]+'.csv')
    except:
        print('Manual ispection needed :' , name[index])
    #data.to_csv('temp_v2.csv')
df_updated = df_select.copy()
df_updated.insert(5 , 'num_obs' , num_obs)

In [33]:
df_updated.to_csv('../source_list/CV_updated.csv')